# 第三章 线性神经网络

## 3.1 线性回归

### 3.1.1 线性回归的基本元素

#### 线性模型
* 单样本 $\hat{y} = w_1x_1 + ... + w_dx_d + b$

* 单样本向量表示 $\hat{y} = {\textbf{w}}^T\textbf{x} + b$

* 多样本 $\hat{\textbf{y}} = \textbf{X}\textbf{w} + b$

#### 损失函数
量化目标的实际值与预测值之间的差距
* 单样本 $l^{(i)}(\textbf{w}, b) = \frac{1}{2}({\hat{y}}^{(i)}-y^{(i)})^2$

* 多样本 $L(\textbf{w},b) = \frac{1}{n}\sum\limits_{i=1}^{n}l^{(i)}(\textbf{w}, b) = \frac{1}{n}\sum\limits_{i=1}^{n}\frac{1}{2}({\textbf{w}}^T{\\textbf{x}}^{(i)}+b-y^{(i)})^2$

训练模型的目标：寻找一组参数$(\textbf{w}^*,b^*)$，这组参数能**最小化在所有训练样本上的总损失**，即：

$$\textbf{w}^*,b^* = \underset{\textbf{w},b}{argmin} L(\textbf{w},b)$$

#### 解析解
线性回归存在解析解：
1. 将偏置$b$合并到参数$\textbf{w}$中，即在包含所有参数的矩阵中附加一列
2. 预测问题变为最小化${||\textbf{y}-\textbf{Xw}||}^2$
3. 损失关于$\textbf{w}$导数为0处为解析解：${\textbf{w}}^*=({\textbf{X}}^T\textbf{X})^{-1}{\textbf{X}}^T{\textbf{y}}$

#### 随机梯度下降
并不是所有问题都存在解析解，所以需要随机梯度下降

* 梯度下降 (gradient descent)
    * 不断在损失函数递减的方向上（关于模型参数的导数）更新参数来降低误差
    * 性能问题：每一次更新参数之前都需要遍历整个数据集
* 随机梯度下降 (minibatch stochastic gradient descent)
    * 在每次需要计算更新的时候抽取一小批样本
    * 更新过程的数学表示：
      $$(\textbf{w},b)\leftarrow(\textbf{w},b)-\frac{\eta}{|B|}\sum\limits_{i\in{B}}\partial_{(\textbf{w},b)}l^{(i)}(\textbf{w}, b)$$
        * $\eta$: 学习率
        * $|B|$: 每个小批量中的样本数，即批量大小
        * $\eta$和$|B|$需手动指定，称为**超参数（hyperparameter）**，**调参**是选择超参数的过程
    * 展开：
      $$\textbf{w}\leftarrow\textbf{w}-\frac{\eta}{|B|}\sum\limits_{i\in{B}}\partial_{\textbf{w}}l^{(i)}(\textbf{w}, b)=\textbf{w}-\frac{\eta}{|B|}\sum\limits_{i\in{B}}{\textbf{x}}^{(i)}({\textbf{w}}^T{\textbf{x}}^{(i)}+b-y^{(i)})$$

      $$b\leftarrow b-\frac{\eta}{|B|}\sum\limits_{i\in{B}}\partial_bl^{(i)}(\textbf{w}, b)=b-\frac{\eta}{|B|}\sum\limits_{i\in{B}}({\textbf{w}}^T{\textbf{x}}^{(i)}+b-y^{(i)})$$

### 3.1.2 矢量化加速
对计算进行矢量化从而利用**线性代数库**，而不是使用Python for循环

示例：

In [9]:
%matplotlib inline
import math
import time
import numpy as np
import torch

class Timer: #@save
    def __init__(self):
        self.times = []
        self.start()

    def start(self):
        """启动计时器"""
        self.tik = time.time()

    def stop(self):
        """停止计时器并将时间记录在列表中"""
        self.times.append(time.time() - self.tik)
        return self.times[-1]

    def avg(self):
        """返回平时时间"""
        return sum(self.times) / len(self.times)

    def sum(self):
        """返回时间总和"""
        return sum(self.times)

    def cumsum(self):
        """返回累计时间"""
        return np.array(self.times).cumsum().tolist()

n = 10000
a = torch.ones([n])
b = torch.ones([n])
c = torch.zeros(n)
timer = Timer()
for i in range(n):
    c[i] = a[i] + b[i]
f'{timer.stop():.5f} sec'

'0.25728 sec'

In [8]:
timer.start()
d = a + b
f'{timer.stop():.5f} sec'

'0.00057 sec'

### 3.1.2 正态分布与平方损失

#### 正态分布 normal distribution
* 也称为高斯分布（Gaussian distribution）
* 随机变量$x$具有均值$\mu$和方差${\sigma}^2$，其正态分布概率密度函数如下：
$$p(x)=\frac{1}{\sqrt{2\pi{\sigma}^2}}\text{exp}(-\frac{1}{2{\sigma}^2}(x-\mu)^2)$$

In [10]:
def normal(x, mu, sigma):
    p = 1 / math.sqrt(2 * math.pi * sigma**2)
    return p * np.exp(-0.5 / sigma**2 * (x - mu)**2)

In [11]:
# 再次使用numpy进行可视化
x = np.arange(-7, 7, 0.01)

# 均值和标准差对
params = [(0, 1), (0, 2), (3, 1)]